## Schema 2: Prescriptions

![Cube 2](img/Cube2.png)

In [2]:
# Libraries and connections

require 'sequel'

DB_MIMIC = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['mimiciii']
                          )

DB_OLAP = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['olap']
                          )

load '../etl/model.rb'
nil

#### Time dimension table

In [8]:
# Creation
DB_OLAP.drop_table?(:prescription_date)
DB_OLAP.create_table(:prescription_date) do
  DateTime :start_date, primary_key: true
  String :day
  String :month
  String :year
end

# Population
Prescription.all.uniq.each do |prescription|
  time = prescription.startdate
  year = time.year
  month = time.month
  day = time.day
  if DB_OLAP[:prescription_date].where(start_date: time).empty?
    DB_OLAP[:prescription_date].insert(
      start_date: time,
      day: "#{year}-#{month}-#{day}",
      month: "#{year}-#{month}",
      year: year.to_s
    )
  end
end

nil

#### Drug dimension table

Note that mimic has multiple fomats for ndc codes, and the community didn't find a method to normalize it.
For simplicity we chose to adopt the 5-4-2 format for all the codes. See the [GitHub issue](https://github.com/MIT-LCP/mimic-code/issues/132) and the [ndc conversion from 10 to 11 digits](https://phpa.health.maryland.gov/OIDEOR/IMMUN/Shared%20Documents/Handout%203%20-%20NDC%20conversion%20to%2011%20digits.pdf).

In [16]:
# Creation
DB_OLAP.drop_table?(:prescription_drug)
DB_OLAP.create_table(:prescription_drug) do
  String :ndc_code, primary_key: true
  String :ndc_product
  String :ndc_labeler
  String :ndc_package
end

# Population
Prescription.all.uniq.each do |prescription|
  ndc_code = prescription.ndc
  ndc_labeler = '00000'
  ndc_product = ndc_labeler + '-0000'
  ndc_package = ndc_product + '-00'
  if ndc_code == nil
    ndc_code = '0'
  end
  if ndc_code.length >= 9
    ndc_labeler = ndc_code[0..4]
    ndc_product = ndc_labeler + '-' + ndc_code[5..8]
    ndc_package = ndc_product + '-' + ndc_code[9..10]
  end
  if DB_OLAP[:prescription_drug].where(ndc_code: ndc_code).empty?
    DB_OLAP[:prescription_drug].insert(
      ndc_code: ndc_code,
      ndc_labeler: ndc_labeler,
      ndc_product: ndc_product,
      ndc_package: ndc_package
    )
  end
end

nil

#### Drug type dimension table

In [17]:
# Creation
DB_OLAP.drop_table?(:prescription_drug_type)
DB_OLAP.create_table(:prescription_drug_type) do
  String :drug_type, primary_key: true
end

# Population
Prescription.all.uniq.each do |prescription|
  type = prescription.drug_type
  if DB_OLAP[:prescription_drug_type].where(drug_type: type).empty?
    DB_OLAP[:prescription_drug_type].insert(
      drug_type: type
    )
  end
end

nil

#### Fact table

In [18]:
def age_in_completed_years (bd, d)
    # Difference in years, less one if you have not had a birthday this year.
    a = d.year - bd.year
    a = a - 1 if (
         bd.month >  d.month or 
        (bd.month >= d.month and bd.day > d.day)
    )
    a
end

:age_in_completed_years

In [19]:
# Creation
DB_OLAP.drop_table?(:prescriptions)
DB_OLAP.create_table(:prescriptions) do
  Integer :row_id, primary_key: true
  Integer :duration
  Integer :patient_age
  
  DateTime :start_date
  String :ndc_code
  String :drug_type
end

# Population
Prescription.all.uniq.each do |prescription|
  
  duration = nil
  if not prescription.enddate.nil?
    duration = (prescription.enddate - prescription.startdate).to_i / ( 3600 * 24 )  # Calculte elapsed days
  end
  DB_OLAP[:prescriptions].insert(
    row_id: prescription.row_id,
    duration: duration,
    patient_age: age_in_completed_years(prescription.patient.dob, prescription.startdate),
    
    start_date: prescription.startdate,
    ndc_code: prescription.ndc,
    drug_type: prescription.drug_type
  )
  
end

nil

## XML implementation